In [17]:
import cv2 as cv
import numpy as np
from keras.models import load_model
from statistics import mode
from os.path import dirname, join
from helper import get_labels
from helper import detect_faces
from helper import draw_text
from helper import draw_bounding_box
from helper import apply_offsets
from helper import load_detection_model
from helper import preprocess_input

USE_WEBCAM = False

In [18]:
# image detection
# detection_model_path = '../models/detection/res10_300x300_ssd_iter_140000.caffemodel'
# prototxt_path = join(dirname('.'), 'deploy.prototxt.txt')

# video detection
emotion_model_path = '../models/emotion/XCEPTION_107-0.66.hdf5'
emotion_labels = get_labels()

In [19]:
# hyper-parameters for bounding boxes shape
frame_window = 10
emotion_offsets = (20, 40)

# loading models
#net = cv2.dnn.readNetFromCaffe(prototxt_path, detection_model_path)
net = cv.dnn.readNetFromCaffe('../models/detection/deploy.prototxt.txt', '../models/detection/res10_300x300_ssd_iter_140000.caffemodel')
emotion_classifier = load_model(emotion_model_path, compile=False)

# getting input model shapes for inference
emotion_target_size = emotion_classifier.input_shape[1:3]

# starting lists for calculating modes
emotion_window = []

In [20]:
# start video streaming
cv2.namedWindow('window_frame')
video_capture = cv2.VideoCapture(0)
print("Stream Started")

Stream Started


In [21]:
cap = None
if (USE_WEBCAM == True):
    cap = cv2.VideoCapture(0)
else:
    print("Loading Dinner Video")
    cap = cv2.VideoCapture('test_images/trailer.mp4')
    
print("Loaded Video")

Loading Dinner Video
Loaded Video


In [22]:
print("Reading Frame")
while cap.isOpened(): # True:
    
    print("Reading Frame")
    ret, bgr_image = cap.read()

    #bgr_image = video_capture.read()[1]

    print("Setting GRAY and RGB Images")
    gray_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2GRAY)
    rgb_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)

    print("Set Blob as Modified Image")
    (h, w) = bgr_image.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(bgr_image, (300, 300)), 1.0,
                                 (300, 300), (104.0, 177.0, 123.0))
    
    print("Detecting Faces")
    net.setInput(blob)
    detections = net.forward()

    print("Iterating Over " + str(detections.shape[2]) + " Detections")
    for i in range(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with the prediction
        confidence = detections[0, 0, i, 2]
        
        # use detections passing a confidence threshold
        if confidence > 0.240:
            # calculate coordinates for detection box
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            
            # calculate emotion predictions
            try:
                gray_face = cv2.resize(gray_face, (emotion_target_size)) # resize the image to emotion image size
            except:
                continue
            
            gray_face = preprocess_input(gray_face, True) # convert pixel values to between -1 and 1
            gray_face = np.expand_dims(gray_face, 0) # add space after each pixel coordinate
            gray_face = np.expand_dims(gray_face, -1) # add new line after each pixel coordinate
            emotion_prediction = emotion_rec.predict(gray_face) # use model to predict face emotion
            emotion_probability = np.max(emotion_prediction) # assign the mostly like emotion values
            emotion_label_arg = np.argmax(emotion_prediction) # assign most occuring emotion value
            emotion_text = emotion_labels[emotion_label_arg] # # assign corresponding emotion text
            emotion_window.append(emotion_text) # append the emotion text to array

            if len(emotion_window) > frame_window: # if number of appended emotions is greater than 10
                emotion_window.pop(0) # remove the oldest predicted emotion
            try:
                emotion_mode = mode(emotion_window) # create array of most commonly occuring emotions
            except:
                continue

            # setup emotion colors
            if emotion_text == 'angry':
                color = emotion_probability * np.asarray((255, 0, 0))
            elif emotion_text == 'sad':
                color = emotion_probability * np.asarray((0, 0, 255))
            elif emotion_text == 'happy':
                color = emotion_probability * np.asarray((255, 255, 0))
            elif emotion_text == 'surprise':
                color = emotion_probability * np.asarray((0, 255, 255))
            else:
                color = emotion_probability * np.asarray((0, 255, 0))

            color = color.astype(int)
            color = color.tolist()

            # draw box and confidence
            text = "{:.2f}%".format(confidence * 100)
            y = startY - 10 if startY - 10 > 10 else startY + 10
            cv2.rectangle(bgr_image, (startX, startY), (endX, endY), (0, 0, 255), 2)
            cv2.putText(bgr_image, text, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
            draw_text(box, rgb_image, emotion_mode, color, 0, -45, 1, 1)
    
    cv2.imshow('window_frame', bgr_image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Reading Frame
Reading Frame
Setting GRAY and RGB Images
Set Blob as Modified Image
Detecting Faces
Iterating Over 165 Detections
Reading Frame
Setting GRAY and RGB Images
Set Blob as Modified Image
Detecting Faces
Iterating Over 163 Detections
Reading Frame
Setting GRAY and RGB Images
Set Blob as Modified Image
Detecting Faces
Iterating Over 163 Detections
Reading Frame
Setting GRAY and RGB Images
Set Blob as Modified Image
Detecting Faces
Iterating Over 170 Detections
Reading Frame
Setting GRAY and RGB Images
Set Blob as Modified Image
Detecting Faces
Iterating Over 171 Detections
Reading Frame
Setting GRAY and RGB Images
Set Blob as Modified Image
Detecting Faces
Iterating Over 175 Detections
Reading Frame
Setting GRAY and RGB Images
Set Blob as Modified Image
Detecting Faces
Iterating Over 170 Detections
Reading Frame
Setting GRAY and RGB Images
Set Blob as Modified Image
Detecting Faces
Iterating Over 162 Detections
Reading Frame
Setting GRAY and RGB Images
Set Blob as Modified Ima